### Ranker

$$
\sum_{i=1}^n \text{IDF}(q_i) \frac{f(q_i, D) \times (k_1 + 1)}{f(q_i, D) + k_1 (1 - b + b\frac{|D|}{\text{avgdl}})}
$$

where:
- $Q$: query  
- $D$: $|D|$ is document length  
- $\text{avgdl}$: average length of a document  
- $k_1$, $b$: free parameters  
- $f(q_i, D)$: the number of times that keyword $q_i$ appears in the document $D$  
- $\text{IDF}(q_i)$: the inverse document frequency  

$$
\text{IDF}(q_i) = \ln(1 + \frac{N - n(q_i) + 0.5}{n(q_i) + 0.5})
$$

where:
- $N$: the number of documents  
- $n(q_i)$: the number of documents containing $q_i$

[reference](https://www.alexmolas.com/2024/02/05/a-search-engine-in-80-lines.html)

In [30]:
from __future__ import annotations

from collections import defaultdict
from math import log
import string
from typing import TypeAlias

In [31]:
UrlScores: TypeAlias = dict[str, float]

def normalize_string(input_string: str) -> str:
    """구두점 제거, 소문자 변환, 중복 공백 정리"""
    translation_table = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    string_without_punc = input_string.translate(translation_table)
    return ' '.join(string_without_punc.split()).lower()


def update_url_scores(old: UrlScores, new: UrlScores) -> UrlScores:
    """새 점수를 기존 딕셔너리에 합산"""
    for url, score in new.items():
        old[url] = old.get(url, 0.0) + score
    return old

In [32]:
class SearchEngine:
    def __init__(self, k1: float = 1.5, b: float = 0.75):
        # 역인덱스: 단어 → {URL: 빈도수}
        self._index: defaultdict[str, defaultdict[str, int]] = defaultdict(lambda: defaultdict(int))
        # 문서 저장: URL → 내용 (길이 계산용)
        self._documents: dict[str, str] = {}
        self.k1 = k1
        self.b = b

    @property
    def number_of_documents(self) -> int:
        return len(self._documents)

    @property
    def avdl(self) -> float:
        """전체 문서의 평균 단어 길이"""
        if not self._documents:
            return 0.0
        total_words = sum(len(content.split()) for content in self._documents.values())
        return total_words / self.number_of_documents

    def idf(self, kw: str) -> float:
        """Inverse Document Frequency"""
        N = self.number_of_documents
        kw_norm = normalize_string(kw)
        n_kw = len(self._index[kw_norm])
        return log((N - n_kw + 0.5) / (n_kw + 0.5) + 1)

    def bm25(self, kw: str) -> UrlScores:
        """하나의 키워드에 대한 BM25 점수 계산"""
        kw_norm = normalize_string(kw)
        if kw_norm not in self._index:
            return {}

        result: UrlScores = {}
        idf_score = self.idf(kw)
        avdl = self.avdl

        for url, freq in self._index[kw_norm].items():
            doc_length = len(self._documents[url].split())
            numerator = freq * (self.k1 + 1)
            denominator = freq + self.k1 * (1 - self.b + self.b * doc_length / avdl)
            result[url] = idf_score * (numerator / denominator)

        return result

    def search(self, query: str, top_n: int = 5) -> list[tuple[str, float]]:
        """쿼리 검색 → 상위 top_n 결과 반환 (점수 내림차순)"""
        keywords = normalize_string(query).split()
        url_scores: UrlScores = {}

        for kw in keywords:
            kw_scores = self.bm25(kw)
            update_url_scores(url_scores, kw_scores)

        ranked = sorted(url_scores.items(), key=lambda x: x[1], reverse=True)
        return ranked[:top_n]

    def index(self, url: str, content: str) -> None:
        """단일 문서 인덱싱"""
        self._documents[url] = content
        words = normalize_string(content).split()
        for word in words:
            self._index[word][url] += 1

    def bulk_index(self, documents: list[tuple[str, str]]) -> None:
        """여러 문서 일괄 인덱싱"""
        for url, content in documents:
            self.index(url, content)

In [33]:
# 검색 엔진 생성
engine = SearchEngine()

# 샘플 문서
sample_docs = [
    ("https://example.com/post1", "Python is a great programming language. Python is easy to learn."),
    ("https://example.com/post2", "Machine learning with Python is very popular these days."),
    ("https://example.com/post3", "JavaScript is used for web development. Python can also be used on the web."),
    ("https://example.com/post4", "I love coding in Python every day."),
]

engine.bulk_index(sample_docs)

print(f"인덱싱된 문서: {engine.number_of_documents}개")
print(f"평균 문서 길이: {engine.avdl:.2f} 단어\n")

# 검색 테스트
results = engine.search("Python programming", top_n=4)
for rank, (url, score) in enumerate(results, 1):
    print(f"{rank}. {url} (점수: {score:.4f})")

인덱싱된 문서: 4개
평균 문서 길이: 10.25 단어

1. https://example.com/post1 (점수: 1.3126)
2. https://example.com/post4 (점수: 0.1229)
3. https://example.com/post2 (점수: 0.1115)
4. https://example.com/post3 (점수: 0.0905)


### TF-IDF

[wikidocs](https://wikidocs.net/31698)

In [34]:
import string
import math
import csv

In [35]:
STOP_WORDS = {
    'a', 'an', 'and', 'the', 'or', 'of', 'to', 'in', 'for', 'on', 'with', 'at', 'by',
    'i', 'you', 'he', 'she', 'it', 'we', 'they', 'me', 'him', 'her', 'us', 'them',
    'my', 'your', 'his', 'its', 'our', 'their', 'this', 'that', 'these', 'those',
    'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'do',
    'does', 'did', 'will', 'would', 'shall', 'should', 'may', 'might', 'must',
    'can', 'could', 'as', 'but', 'if', 'because', 'until', 'while', 'about',
    'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above',
    'below', 'from', 'up', 'down', 'out', 'off', 'over', 'under', 'again', 'further',
    'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any',
    'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor',
    'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very'
}

def preprocess_text(text):
    text = text.lower()
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    tokens = text.split()
    tokens = [token for token in tokens if token not in STOP_WORDS and token.strip() != '']
    tokens = [stem_token(token) for token in tokens]
    return tokens

def stem_token(token):
    suffixes = ['ing', 'ly', 'ed', 'es', 's']
    for suffix in suffixes:
        if token.endswith(suffix) and len(token) > len(suffix) + 1:
            token = token[:-len(suffix)]
    return token

In [36]:
def build_inverted_index(documents, index_file='inverted_index.csv'):
    inverted_index = defaultdict(list)
    for doc_id, doc in enumerate(documents):
        tokens = preprocess_text(doc)
        term_positions = defaultdict(list)
        for pos, term in enumerate(tokens):
            term_positions[term].append(pos)
        for term, positions in term_positions.items():
            inverted_index[term].append((doc_id, positions))
    
    # CSV 저장
    with open(index_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['term', 'doc_id', 'positions'])
        for term, doc_info in inverted_index.items():
            for doc_id, positions in doc_info:
                writer.writerow([term, doc_id, str(positions)])
    
    return inverted_index

In [37]:
def calculate_tfidf(documents, inverted_index):
    num_docs = len(documents)
    tfidf = defaultdict(dict)
    doc_lengths = [len(preprocess_text(doc)) for doc in documents]
    doc_freq = {term: len(entries) for term, entries in inverted_index.items()}
    
    for term, entries in inverted_index.items():
        idf = math.log(num_docs / (doc_freq[term] + 1))
        for doc_id, positions in entries:
            tf = len(positions) / doc_lengths[doc_id] if doc_lengths[doc_id] > 0 else 0
            tfidf[doc_id][term] = tf * idf
    
    return tfidf

In [38]:
def search(query, documents, inverted_index, tfidf_scores, top_n=5):
    query_terms = preprocess_text(query)
    if not query_terms:
        return []
    
    relevant_docs = set()
    for term in query_terms:
        if term in inverted_index:
            for doc_id, _ in inverted_index[term]:
                relevant_docs.add(doc_id)
    
    scores = []
    for doc_id in relevant_docs:
        score = sum(tfidf_scores.get(doc_id, {}).get(term, 0) for term in query_terms)
        score /= len(query_terms) or 1
        scores.append((doc_id, score))
    
    scores.sort(key=lambda x: x[1], reverse=True)
    results = [(documents[doc_id], score) for doc_id, score in scores[:top_n] if score > 0]
    return results

In [39]:
# 샘플 문서 (이전처럼 위키나 블로그에서 가져온 텍스트로 대체 가능)
documents = [
    "Python is a versatile programming language used for web development and data analysis.",
    "Machine learning algorithms can predict outcomes based on historical data.",
    "Search engines like Google use complex ranking systems to deliver relevant results.",
    "Bing integrates AI to improve search accuracy and user experience.",
    "Building a search engine involves crawling, indexing, and querying large datasets."
]

inverted_index = build_inverted_index(documents)
tfidf_scores = calculate_tfidf(documents, inverted_index)

# 검색 테스트
results = search("search engine python", documents, inverted_index, tfidf_scores)
for i, (doc, score) in enumerate(results, 1):
    print(f"{i}. Score: {score:.4f} - {doc}")

1. Score: 0.0422 - Building a search engine involves crawling, indexing, and querying large datasets.
2. Score: 0.0339 - Python is a versatile programming language used for web development and data analysis.
3. Score: 0.0093 - Bing integrates AI to improve search accuracy and user experience.
4. Score: 0.0068 - Search engines like Google use complex ranking systems to deliver relevant results.


### Scrap sites

In [43]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from collections import deque
import time
import re


def is_valid_url(url: str, base_domain: str) -> bool:
    """
    Check if a URL is valid and belongs to the same domain as the start URL.
    Also skips non-HTML files like images, PDFs, etc.
    """
    parsed = urlparse(url)
    return (
        parsed.scheme in ('http', 'https') and
        parsed.netloc == base_domain and
        not parsed.path.lower().endswith(('.pdf', '.jpg', '.jpeg', '.png', '.gif', '.zip', '.exe', '.js', '.css'))
    )


def clean_content(html: str) -> str | None:
    """
    Extract clean, readable text from HTML.
    Removes scripts, styles, navigation, footers, etc., and skips pages with too little content.
    """
    soup = BeautifulSoup(html, 'html.parser')
    
    # Remove unwanted elements that usually don't contain useful content
    for tag in soup(["script", "style", "nav", "footer", "header", "aside", "iframe", "noscript"]):
        tag.decompose()
    
    text = soup.get_text(separator=' ', strip=True)
    
    # Skip pages with very little text (e.g., login pages, error pages)
    if len(text.split()) < 20:
        return None
    
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text)
    return text.strip()


def auto_crawl_and_index(
    engine,
    start_url: str,
    max_pages: int = 50,
    max_depth: int = 5,
    delay: float = 1.0
) -> None:
    """
    Fully automatic web crawler with indexing.
    Starts from a seed URL and follows links breadth-first (BFS), just like Google.
    Only crawls within the same domain and respects basic politeness rules.
    """
    # Ensure URL has protocol
    if not start_url.startswith(('http://', 'https://')):
        start_url = 'https://' + start_url
    
    base_domain = urlparse(start_url).netloc
    visited = set()
    queue = deque([(start_url, 0)])  # (url, current_depth)

    print(f"Starting automatic crawl from: {start_url}")
    print(f"Domain restricted to: {base_domain}")
    print(f"Max pages: {max_pages}, Max depth: {max_depth}\n")

    while queue and len(visited) < max_pages:
        current_url, depth = queue.popleft()

        # Skip if already visited or too deep
        if current_url in visited or depth > max_depth:
            continue

        print(f"[{len(visited)+1}/{max_pages}] Crawling: {current_url} (depth: {depth})")

        try:
            # Polite User-Agent (important to avoid being blocked)
            headers = {
                'User-Agent': 'MyPersonalSearchBot/1.0 (+https://yourwebsite.com/bot-info)'
            }
            response = requests.get(current_url, headers=headers, timeout=15)
            response.raise_for_status()

            # Skip non-HTML content
            content_type = response.headers.get('Content-Type', '')
            if 'text/html' not in content_type:
                print(f"Not HTML, skipping")
                visited.add(current_url)
                continue

            content = clean_content(response.text)
            if content is None:
                print(f"  → Too little content, skipping")
                visited.add(current_url)
                continue

            # Index the page into your SearchEngine
            engine.index(current_url, content)
            visited.add(current_url)

            # Extract and enqueue new links
            soup = BeautifulSoup(response.text, 'html.parser')
            new_links = 0
            for link in soup.find_all('a', href=True):
                next_url = urljoin(current_url, link['href'])

                # Add only valid, unvisited internal links
                if (is_valid_url(next_url, base_domain) and
                    next_url not in visited and
                    next_url not in [item[0] for item in queue]):
                    
                    queue.append((next_url, depth + 1))
                    new_links += 1

            print(f"> Indexed successfully | Found {new_links} new links")

            # Be respectful: delay between requests
            time.sleep(delay)

        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")
        except Exception as e:
            print(f"Unexpected error: {e}")

    print(f"\nCrawling finished! Total indexed pages: {len(visited)}")
    print("Your search engine is now ready to query these pages!")


# ==================== Example Usage ====================

# Create your SearchEngine instance (make sure the class is defined earlier)
engine = SearchEngine()

# Start crawling from a seed page
# Great test sites: small blogs, essay collections, documentation sites
auto_crawl_and_index(
    engine,
    start_url="https://www.paulgraham.com/articles.html",  # Paul Graham's essays (excellent content)
    max_pages=30,
    max_depth=3,
    delay=1.5  # Be polite – 1.5 seconds between requests
)

# Test searches
print("\n" + "=" * 60)
results = engine.search("startup funding", top_n=5)
for rank, (url, score) in enumerate(results, 1):
    print(f"{rank}. [{score:.4f}] {url}")

print("\n")
results = engine.search("programming language design")
for rank, (url, score) in enumerate(results, 1):
    print(f"{rank}. [{score:.4f}] {url}")

Starting automatic crawl from: https://www.paulgraham.com/articles.html
Domain restricted to: www.paulgraham.com
Max pages: 30, Max depth: 3

[1/30] Crawling: https://www.paulgraham.com/articles.html (depth: 0)
> Indexed successfully | Found 230 new links
[2/30] Crawling: https://www.paulgraham.com/index.html (depth: 1)
> Indexed successfully | Found 0 new links
[3/30] Crawling: https://www.paulgraham.com/greatwork.html (depth: 1)
> Indexed successfully | Found 29 new links
[4/30] Crawling: https://www.paulgraham.com/kids.html (depth: 1)
> Indexed successfully | Found 1 new links
[5/30] Crawling: https://www.paulgraham.com/selfindulgence.html (depth: 1)
> Indexed successfully | Found 0 new links
[6/30] Crawling: https://www.paulgraham.com/field.html (depth: 1)
> Indexed successfully | Found 1 new links
[7/30] Crawling: https://www.paulgraham.com/goodwriting.html (depth: 1)
> Indexed successfully | Found 3 new links
[8/30] Crawling: https://www.paulgraham.com/do.html (depth: 1)
> Indexe

In [44]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from collections import deque
import time
import re
from typing import Set
from urllib.robotparser import RobotFileParser
from urllib.request import urlopen
import urllib.parse


def is_valid_url(url: str, allowed_domains: Set[str] | None = None) -> bool:
    """
    Validate URL: proper scheme, not a file, and (optionally) within allowed domains.
    """
    parsed = urlparse(url)
    if parsed.scheme not in ('http', 'https'):
        return False
    if parsed.path.lower().endswith(('.pdf', '.jpg', '.jpeg', '.png', '.gif', '.zip', '.exe', '.js', '.css', '.ico')):
        return False
    if allowed_domains is not None and parsed.netloc not in allowed_domains:
        return False
    return True


def clean_content(html: str) -> str | None:
    """
    Extract clean readable text from HTML.
    Removes noise (scripts, nav, footer, etc.) and skips pages with too little content.
    """
    soup = BeautifulSoup(html, 'html.parser')
    
    # Remove non-content elements
    for tag in soup(["script", "style", "nav", "footer", "header", "aside", 
                     "iframe", "noscript", "form", "button", "input"]):
        tag.decompose()
    
    text = soup.get_text(separator=' ', strip=True)
    
    # Skip pages with almost no real content
    if len(text.split()) < 30:
        return None
    
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text)
    return text.strip()


def get_robots_parser(domain: str, user_agent: str) -> RobotFileParser:
    """
    Fetch and parse robots.txt for a given domain.
    Returns a RobotFileParser object.
    """
    rp = RobotFileParser()
    robots_url = urllib.parse.urljoin(domain, '/robots.txt')
    
    try:
        with urlopen(robots_url, timeout=10) as response:
            rp.parse(response.read().decode('utf-8').splitlines())
    except Exception as e:
        print(f"Could not fetch robots.txt for {domain}: {e}")
        # If robots.txt is unavailable, assume all is allowed (common practice)
    
    return rp


def auto_crawl_and_index(
    engine,
    seed_urls,
    max_pages: int = 100,
    max_depth: int = 6,
    delay: float = 1.0,
    global_mode: bool = False
) -> None:
    """
    Advanced automatic web crawler with two modes:
    
    - global_mode=False (default): Crawl within the initial domains only (deep dive one site)
    - global_mode=True: Allow discovering and crawling NEW domains (true web-wide search engine behavior)
    
    Respects robots.txt for each domain.
    """
    # Convert single string to list if needed
    if isinstance(seed_urls, str):
        seed_urls = [seed_urls]
    
    # Normalize seed URLs
    normalized_seeds = []
    for url in seed_urls:
        if not url.startswith(('http://', 'https://')):
            url = 'https://' + url
        normalized_seeds.append(url)
    
    seed_urls = normalized_seeds
    
    # Determine allowed domains
    if global_mode:
        allowed_domains = None  # No restriction - can explore the entire web
        print("GLOBAL MODE ACTIVATED: Will discover and crawl new domains!")
    else:
        allowed_domains = {urlparse(url).netloc for url in seed_urls}
        print(f"DOMAIN-LOCKED MODE: Limited to {len(allowed_domains)} starting domain(s)")
    
    print(f"Seed URLs: {seed_urls}")
    print(f"Max pages: {max_pages} | Max depth: {max_depth} | Delay: {delay}s\n")

    visited = set()
    queue = deque([(url, 0) for url in seed_urls])  # (url, depth)

    discovered_domains = set(urlparse(url).netloc for url in seed_urls)
    robots_parsers = {}  # Cache: domain -> RobotFileParser

    user_agent = 'MyGlobalSearchBot/1.0 (+https://yourwebsite.com/bot)'

    while queue and len(visited) < max_pages:
        current_url, depth = queue.popleft()

        if current_url in visited or depth > max_depth:
            continue

        print(f"[{len(visited)+1}/{max_pages}] Crawling: {current_url} (depth: {depth})")

        # Get domain and check robots.txt
        parsed_url = urlparse(current_url)
        domain = f"{parsed_url.scheme}://{parsed_url.netloc}/"
        
        if domain not in robots_parsers:
            robots_parsers[domain] = get_robots_parser(domain, user_agent)
        
        rp = robots_parsers[domain]
        
        if not rp.can_fetch(user_agent, current_url):
            print(f"Blocked by robots.txt: {current_url}")
            visited.add(current_url)  # Mark as visited to avoid retrying
            continue

        try:
            headers = {'User-Agent': user_agent}
            response = requests.get(current_url, headers=headers, timeout=15)
            response.raise_for_status()

            if 'text/html' not in response.headers.get('Content-Type', ''):
                visited.add(current_url)
                continue

            content = clean_content(response.text)
            if content is None:
                visited.add(current_url)
                continue

            # Index the page
            engine.index(current_url, content)
            visited.add(current_url)

            current_domain = parsed_url.netloc
            if current_domain not in discovered_domains and global_mode:
                discovered_domains.add(current_domain)
                print(f"NEW DOMAIN DISCOVERED: {current_domain}")

            # Extract links
            soup = BeautifulSoup(response.text, 'html.parser')
            new_links = 0
            for link in soup.find_all('a', href=True):
                next_url = urljoin(current_url, link['href'])

                if (is_valid_url(next_url, allowed_domains) and
                    next_url not in visited and
                    next_url not in [item[0] for item in queue]):

                    queue.append((next_url, depth + 1))
                    new_links += 1

            print(f"Indexed | +{new_links} new links queued")

            time.sleep(delay)

        except requests.exceptions.RequestException as e:
            print(f"Request error: {e}")
        except Exception as e:
            print(f"Error: {e}")

    print(f"\nCrawling complete! Indexed {len(visited)} pages.")
    if global_mode:
        print(f"Discovered {len(discovered_domains)} different domains.")
    print("Your personal search engine now knows more of the web!")


# ==================== USAGE EXAMPLES ====================

engine = SearchEngine()  # Make sure your SearchEngine class is defined

# 1. Single site deep crawl (original behavior)
print("Example 1: Deep crawl of one site")
auto_crawl_and_index(
    engine,
    seed_urls="https://www.paulgraham.com/articles.html",
    max_pages=40,
    max_depth=4,
    delay=1.5,
    global_mode=False
)

# 2. Global web exploration mode - discovers new sites!
print("\n" + "="*70)
print("Example 2: GLOBAL MODE - Exploring the open web")
auto_crawl_and_index(
    engine,
    seed_urls=[
        "https://news.ycombinator.com",
        "https://www.lesswrong.com",
        "https://www.paulgraham.com"
    ],
    max_pages=80,
    max_depth=5,
    delay=2.0,          # Be extra polite in global mode
    global_mode=True   # This is the key!
)

# Test your growing search engine
print("\n" + "="*70)
results = engine.search("artificial intelligence ethics")
for i, (url, score) in enumerate(results[:5], 1):
    print(f"{i}. [{score:.4f}] {url}")

results = engine.search("best programming books")
for i, (url, score) in enumerate(results[:5], 1):
    print(f"{i}. [{score:.4f}] {url}")

Example 1: Deep crawl of one site
DOMAIN-LOCKED MODE: Limited to 1 starting domain(s)
Seed URLs: ['https://www.paulgraham.com/articles.html']
Max pages: 40 | Max depth: 4 | Delay: 1.5s

[1/40] Crawling: https://www.paulgraham.com/articles.html (depth: 0)
Indexed | +230 new links queued
[2/40] Crawling: https://www.paulgraham.com/index.html (depth: 1)
[3/40] Crawling: https://www.paulgraham.com/greatwork.html (depth: 1)
Indexed | +29 new links queued
[4/40] Crawling: https://www.paulgraham.com/kids.html (depth: 1)
Indexed | +1 new links queued
[5/40] Crawling: https://www.paulgraham.com/selfindulgence.html (depth: 1)
Indexed | +0 new links queued
[6/40] Crawling: https://www.paulgraham.com/field.html (depth: 1)
Indexed | +1 new links queued
[7/40] Crawling: https://www.paulgraham.com/goodwriting.html (depth: 1)
Indexed | +3 new links queued
[8/40] Crawling: https://www.paulgraham.com/do.html (depth: 1)
Indexed | +2 new links queued
[9/40] Crawling: https://www.paulgraham.com/woke.html (

In [48]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse, urlunparse
from collections import deque
import time
import re
from typing import Set, List
from urllib.robotparser import RobotFileParser
from urllib.request import urlopen
import urllib.parse
import xml.etree.ElementTree as ET  # For sitemap XML parsing


def is_valid_url(url: str, allowed_domains: Set[str] | None = None) -> bool:
    """
    Validate URL: proper scheme, not a file type, and optionally restricted to domains.
    """
    parsed = urlparse(url)
    if parsed.scheme not in ('http', 'https'):
        return False
    if parsed.path.lower().endswith(('.pdf', '.jpg', '.jpeg', '.png', '.gif', '.zip', '.exe', '.js', '.css', '.ico')):
        return False
    if allowed_domains is not None and parsed.netloc not in allowed_domains:
        return False
    return True


def clean_content(html: str) -> str | None:
    """
    Extract clean readable text from HTML.
    Removes noise and skips pages with insufficient content.
    """
    soup = BeautifulSoup(html, 'html.parser')
    
    for tag in soup(["script", "style", "nav", "footer", "header", "aside", 
                     "iframe", "noscript", "form", "button", "input"]):
        tag.decompose()
    
    text = soup.get_text(separator=' ', strip=True)
    
    if len(text.split()) < 30:
        return None
    
    text = re.sub(r'\s+', ' ', text)
    return text.strip()


def get_robots_parser(domain: str, user_agent: str) -> RobotFileParser:
    """
    Fetch and parse robots.txt for a domain.
    """
    rp = RobotFileParser()
    robots_url = urllib.parse.urljoin(domain, '/robots.txt')
    
    try:
        with urlopen(robots_url, timeout=10) as response:
            rp.parse(response.read().decode('utf-8').splitlines())
    except Exception:
        pass  # If unavailable, assume allowed
    
    return rp


def parse_sitemap(domain: str, sitemap_url: str | None = None) -> List[str]:
    """
    Parse sitemap.xml to extract URLs.
    If sitemap_url is None, try default '/sitemap.xml'.
    Returns list of URLs from <loc> tags.
    """
    urls = []
    
    if sitemap_url is None:
        sitemap_url = urllib.parse.urljoin(domain, '/sitemap.xml')
    
    try:
        response = requests.get(sitemap_url, timeout=10)
        response.raise_for_status()
        
        root = ET.fromstring(response.text)
        ns = {'sitemap': 'http://www.sitemaps.org/schemas/sitemap/0.9'}
        
        for loc in root.findall('.//sitemap:loc', ns):
            urls.append(loc.text.strip())
        
        # If it's a sitemap index, recursively parse sub-sitemaps (simple version, limit recursion)
        if 'sitemapindex' in root.tag:
            for sub_sitemap in root.findall('sitemap:sitemap/sitemap:loc', ns):
                urls.extend(parse_sitemap(domain, sub_sitemap.text.strip()))
        
    except Exception as e:
        print(f"Failed to parse sitemap {sitemap_url}: {e}")
    
    return urls


def get_sitemap_from_robots(rp: RobotFileParser) -> str | None:
    """
    Extract sitemap URL from robots.txt if present.
    """
    sitemaps = rp.site_maps()
    if sitemaps is not None:
        for line in sitemaps:
            if line:
                return line
    return None


def get_homepage_url(url: str) -> str:
    """
    Convert any URL to its homepage.
    """
    parsed = urlparse(url)
    return urlunparse((parsed.scheme, parsed.netloc, '/', '', '', ''))


def auto_crawl_and_index(
    engine,
    initial_seed_urls,
    max_pages: int = 200,
    max_depth: int = 6,
    delay: float = 2.0,
    global_mode: bool = True
) -> list[str]:
    """
    Global web crawler with sitemap parsing.
    Discovers new sites and uses sitemaps to find more URLs within domains.
    Returns list of newly discovered homepages for future seeds.
    """
    if isinstance(initial_seed_urls, str):
        initial_seed_urls = [initial_seed_urls]
    
    seed_urls = []
    for url in initial_seed_urls:
        if not url.startswith(('http://', 'https://')):
            url = 'https://' + url
        seed_urls.append(url)
    
    if global_mode:
        allowed_domains = None
        print("GLOBAL MODE: Exploring freely across domains")
    else:
        allowed_domains = {urlparse(u).netloc for u in seed_urls}
        print(f"DOMAIN-LOCKED MODE: Restricted to initial domains")
    
    print(f"Initial seeds: {len(seed_urls)} URLs")
    print(f"Max pages: {max_pages} | Max depth: {max_depth} | Delay: {delay}s\n")

    visited = set()
    queue = deque([(url, 0) for url in seed_urls])
    discovered_homepages = set()
    robots_parsers = {}
    user_agent = 'MyGlobalSearchBot/1.0 (+https://yourwebsite.com/bot)'

    while queue and len(visited) < max_pages:
        current_url, depth = queue.popleft()
        
        if current_url in visited or depth > max_depth:
            continue

        print(f"[{len(visited)+1}/{max_pages}] Crawling: {current_url} (depth: {depth})")

        parsed_url = urlparse(current_url)
        domain = f"{parsed_url.scheme}://{parsed_url.netloc}/"
        
        if domain not in robots_parsers:
            rp = get_robots_parser(domain, user_agent)
            robots_parsers[domain] = rp
            
            # Check for sitemap in robots.txt and parse it
            sitemap_url = get_sitemap_from_robots(rp)
            if sitemap_url:
                print(f"Found sitemap in robots.txt: {sitemap_url}")
                sitemap_urls = parse_sitemap(domain, sitemap_url)
                for new_url in sitemap_urls:
                    if is_valid_url(new_url, allowed_domains) and new_url not in visited:
                        queue.append((new_url, depth + 1))
                print(f"Added {len(sitemap_urls)} URLs from sitemap")
            else:
                # Fallback: try default sitemap.xml
                default_sitemap_urls = parse_sitemap(domain)
                for new_url in default_sitemap_urls:
                    if is_valid_url(new_url, allowed_domains) and new_url not in visited:
                        queue.append((new_url, depth + 1))
                if default_sitemap_urls:
                    print(f"Added {len(default_sitemap_urls)} URLs from default sitemap")

        rp = robots_parsers[domain]
        if not rp.can_fetch(user_agent, current_url):
            print(f"Blocked by robots.txt: {current_url}")
            visited.add(current_url)
            continue

        try:
            headers = {'User-Agent': user_agent}
            response = requests.get(current_url, headers=headers, timeout=15)
            response.raise_for_status()

            if 'text/html' not in response.headers.get('Content-Type', ''):
                visited.add(current_url)
                continue

            content = clean_content(response.text)
            if content is None:
                visited.add(current_url)
                continue

            engine.index(current_url, content)
            visited.add(current_url)

            current_homepage = get_homepage_url(current_url)
            if current_homepage not in [get_homepage_url(s) for s in seed_urls]:
                discovered_homepages.add(current_homepage)

            soup = BeautifulSoup(response.text, 'html.parser')
            new_links = 0
            for link in soup.find_all('a', href=True):
                next_url = urljoin(current_url, link['href'])

                if (is_valid_url(next_url, allowed_domains) and
                    next_url not in visited and
                    next_url not in [item[0] for item in queue]):

                    queue.append((next_url, depth + 1))
                    new_links += 1

            print(f"Indexed | +{new_links} links queued")
            if current_homepage in discovered_homepages:
                print(f"New site discovered: {current_homepage}")

            time.sleep(delay)

        except requests.exceptions.RequestException as e:
            print(f"Request error: {e}")
        except Exception as e:
            print(f"Error: {e}")

    print(f"\nCrawling complete! Indexed {len(visited)} pages.")
    print(f"Discovered {len(discovered_homepages)} new homepages for future seeds.")
    
    return list(discovered_homepages)


# ==================== USAGE ====================

engine = SearchEngine()

initial_seeds = [
    "https://news.ycombinator.com",
    "https://www.reddit.com",
    "https://en.wikipedia.org/wiki/Main_Page"
]

new_seeds = auto_crawl_and_index(
    engine,
    initial_seed_urls=initial_seeds,
    max_pages=50,
    max_depth=5,
    delay=2.0,
    global_mode=True
)

# Save new seeds
with open("new_seeds.txt", "w") as f:
    for seed in new_seeds:
        f.write(seed + "\n")

print("\nSaved new seeds to new_seeds.txt")

GLOBAL MODE: Exploring freely across domains
Initial seeds: 3 URLs
Max pages: 50 | Max depth: 5 | Delay: 2.0s

[1/50] Crawling: https://news.ycombinator.com (depth: 0)
Failed to parse sitemap https://news.ycombinator.com/sitemap.xml: 404 Client Error: Not Found for url: https://news.ycombinator.com/sitemap.xml
Indexed | +197 links queued
[2/50] Crawling: https://www.reddit.com (depth: 0)
Failed to parse sitemap https://www.reddit.com/sitemap.xml: not well-formed (invalid token): line 40, column 231
Blocked by robots.txt: https://www.reddit.com
[3/50] Crawling: https://en.wikipedia.org/wiki/Main_Page (depth: 0)
Failed to parse sitemap https://en.wikipedia.org/sitemap.xml: 403 Client Error: Forbidden for url: https://en.wikipedia.org/sitemap.xml
Blocked by robots.txt: https://en.wikipedia.org/wiki/Main_Page
[4/50] Crawling: https://news.ycombinator.com/news (depth: 1)
Indexed | +1 links queued
[5/50] Crawling: https://news.ycombinator.com/newest (depth: 1)
Indexed | +194 links queued
[6/